In [1]:
#pragma cling add_include_path("./filament/include")
#pragma cling add_library_path("./filament/lib")
#pragma load("libfilament")

#include <filament/FilamentAPI.h>
#include <filament/Engine.h>

In [2]:
using namespace filament;

In [3]:
Engine *engine = Engine::create();

IncrementalExecutor::executeFunction: symbol '_ZN8filament6Engine7Builder13sharedContextEPv' unresolved while linking function '_GLOBAL__sub_I_cling_module_7'!
You are probably missing the definition of filament::Engine::Builder::sharedContext(void*)
Maybe you need to load the corresponding shared library?
IncrementalExecutor::executeFunction: symbol '_ZN8filament6Engine7BuilderC1Ev' unresolved while linking function '_GLOBAL__sub_I_cling_module_7'!
You are probably missing the definition of filament::Engine::Builder::Builder()
Maybe you need to load the corresponding shared library?
IncrementalExecutor::executeFunction: symbol '_ZN8filament6Engine7Builder8platformEPNS_7backend8PlatformE' unresolved while linking function '_GLOBAL__sub_I_cling_module_7'!
You are probably missing the definition of filament::Engine::Builder::platform(filament::backend::Platform*)
Maybe you need to load the corresponding shared library?
IncrementalExecutor::executeFunction: symbol '_ZN8filament6Engine7Bui